In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%%html
<style>
.cell-output-ipywidget-background {
    background-color: transparent !important;
}
:root {
    --jp-widgets-color: var(--vscode-editor-foreground);
    --jp-widgets-font-size: var(--vscode-editor-font-size);
}  
</style>

In [ ]:
import polars as pl

splits = {'testmini': 'data/testmini-00000-of-00001-725687bf7a18d64b.parquet', 'test': 'data/test-*.parquet'}
df = pl.read_parquet('hf://datasets/AI4Math/MathVista/' + splits['testmini']).sample(fraction=1.0, shuffle=True, seed=42)

In [7]:
val_scenarios = df.head(64).to_dicts()
train_scenarios = df.tail(-64).iter_rows(named=True)

In [8]:
val_scenarios[0]

{'pid': '1',
 'question': "When a spring does work on an object, we cannot find the work by simply multiplying the spring force by the object's displacement. The reason is that there is no one value for the force-it changes. However, we can split the displacement up into an infinite number of tiny parts and then approximate the force in each as being constant. Integration sums the work done in all those parts. Here we use the generic result of the integration.\r\n\r\nIn Figure, a cumin canister of mass $m=0.40 \\mathrm{~kg}$ slides across a horizontal frictionless counter with speed $v=0.50 \\mathrm{~m} / \\mathrm{s}$. It then runs into and compresses a spring of spring constant $k=750 \\mathrm{~N} / \\mathrm{m}$. When the canister is momentarily stopped by the spring, by what distance $d$ is the spring compressed?",
 'image': 'images/1.jpg',
 'decoded_image': {'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x05\xea\x00\x00\x02\xd0\x08\x06\x00\x00\x00\xef\xbd@\xee\x00\x00\n\xaei

In [6]:
df.head().to_dicts()

[{'pid': '1',
  'question': "When a spring does work on an object, we cannot find the work by simply multiplying the spring force by the object's displacement. The reason is that there is no one value for the force-it changes. However, we can split the displacement up into an infinite number of tiny parts and then approximate the force in each as being constant. Integration sums the work done in all those parts. Here we use the generic result of the integration.\r\n\r\nIn Figure, a cumin canister of mass $m=0.40 \\mathrm{~kg}$ slides across a horizontal frictionless counter with speed $v=0.50 \\mathrm{~m} / \\mathrm{s}$. It then runs into and compresses a spring of spring constant $k=750 \\mathrm{~N} / \\mathrm{m}$. When the canister is momentarily stopped by the spring, by what distance $d$ is the spring compressed?",
  'image': 'images/1.jpg',
  'decoded_image': {'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x05\xea\x00\x00\x02\xd0\x08\x06\x00\x00\x00\xef\xbd@\xee\x00\x00\n\